Solve Classification problem (**Sentiment Analysis in NLP**) with RNN (**Deep Learning based Language Model**)

*Duygu Analizi (Sentiment Analysis): Bir cümlenin etitketlenmesi (positive or negative)*
<br>
! Restorant yorumlarını değerlendirme

In [1]:
#!pip install gensim

In [2]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec # metin temsili

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# GPT ile veri üretimi
data = {
    "text": [
        "Yemekler çok lezzetliydi",
        "Servis oldukça hızlıydı",
        "Personel çok güler yüzlüydü",
        "Restoranın ambiyansı çok güzeldi",
        "Yemekler sıcak ve tazeydi",
        "Fiyatlar kaliteye göre uygundu",
        "Tatlılar gerçekten harikaydı",
        "Sunum çok şıktı",
        "Garsonlar çok ilgiliydi",
        "Tekrar gelmeyi düşünüyorum",
        "Yemekler beklentimin üzerindeydi",
        "Servisten çok memnun kaldım",
        "Menü oldukça zengindi",
        "Ortama bayıldık",
        "Lezzetler çok başarılıydı",
        "Her şey kusursuzdu",
        "Yemekler zamanında geldi",
        "Çalışanlar çok nazikti",
        "Restoran oldukça temizdi",
        "Kesinlikle tavsiye ederim",
        "Yemekler çok kötüydü",
        "Servis çok yavaştı",
        "Personel ilgisizdi",
        "Yemekler soğuk geldi",
        "Fiyatlar çok pahalıydı",
        "Lezzet hiç yoktu",
        "Garsonlar çok kabaydı",
        "Siparişler yanlış geldi",
        "Beklemek zorunda kaldık",
        "Restoran çok kirliydi",
        "Yemekler aşırı tuzluydu",
        "Ambiyans çok kötüydü",
        "Yemekler yanmıştı",
        "Porsiyonlar çok küçüktü",
        "Servisten memnun kalmadım",
        "Tatlılar bayattı",
        "Müzik çok rahatsız ediciydi",
        "Yemekler geç geldi",
        "Hiç beğenmedim",
        "Bir daha gelmem",
        "Hizmet kalitesi düşüktü",
        "Yemekler tatsızdı",
        "Garson çağırmak zordu",
        "Mekan çok gürültülüydü",
        "Yemekler beklediğime değmedi",
        "Sunum özensizdi",
        "Masalar kirliydi",
        "Servis çok kötüydü",
        "Deneyim hayal kırıklığıydı",
        "Kesinlikle önermiyorum"
    ],
    "label": [
        "positive","positive","positive","positive","positive",
        "positive","positive","positive","positive","positive",
        "positive","positive","positive","positive","positive",
        "positive","positive","positive","positive","positive",
        "negative","negative","negative","negative","negative",
        "negative","negative","negative","negative","negative",
        "negative","negative","negative","negative","negative",
        "negative","negative","negative","negative","negative",
        "negative","negative","negative","negative","negative",
        "negative","negative","negative","negative","negative"
    ]
}

In [4]:
print(len(data["text"]))
print(len(data["label"]))

from collections import Counter
Counter(data["label"])

50
50


Counter({'positive': 20, 'negative': 30})

In [5]:
df = pd.DataFrame(data)
df.head()

,text,label
0,Yemekler çok lezzetliydi,positive
1,Servis oldukça hızlıydı,positive
2,Personel çok güler yüzlüydü,positive
3,Restoranın ambiyansı çok güzeldi,positive
4,Yemekler sıcak ve tazeydi,positive


In [6]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text"])
sequences = tokenizer.texts_to_sequences(df["text"])
word_index = tokenizer.word_index

print(sequences[0])

[2, 1, 18]


In [7]:
# padding process: farklı uzunluktaki cümleleri aynı düzene getirmek.
maxlen = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=maxlen)
print(X.shape)

(50, 4)


In [8]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["label"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_test

array([[ 0,  0, 43, 44],
       [ 0, 81, 82, 83],
       [ 0,  2, 68, 69],
       [ 0,  0, 10, 95],
       [ 0, 51,  1, 52],
       [ 0, 97, 98, 99],
       [ 0, 11,  1, 62],
       [ 0, 60, 16, 61],
       [ 0,  0,  2, 71],
       [ 0, 15, 54, 55]], dtype=int32)

In [11]:
# Metin temsili
sentences = [text.split() for text in df["text"]]
word2vec_model = Word2Vec(sentences, vector_size=50, window=5, min_count=1)

embedding_dim = 50
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [12]:
embedding_matrix.shape

(101, 50)

In [13]:
# Model

model = Sequential()
# embedding
model.add(Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))

# RNN layer
model.add(SimpleRNN(units=50, return_sequences=False))

# output
model.add(Dense(units=1, activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
# compile
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
# train model

model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.4999 - loss: 0.6891 - val_accuracy: 0.7000 - val_loss: 0.6621
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4902 - loss: 0.6945 - val_accuracy: 0.7000 - val_loss: 0.6622
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5896 - loss: 0.6745 - val_accuracy: 0.7000 - val_loss: 0.6417
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6303 - loss: 0.6550 - val_accuracy: 0.7000 - val_loss: 0.6430
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5682 - loss: 0.6727 - val_accuracy: 0.7000 - val_loss: 0.6438
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6317 - loss: 0.6476 - val_accuracy: 0.7000 - val_loss: 0.6459
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6478 - loss: 0.6402 - val_accuracy: 0.7000 - val_loss: 0.6445
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5025 - loss: 0.6845 - val_accuracy: 0.7000 - val_loss

In [16]:
# evaluation
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred.round()))
print("Classification Report:\n", classification_report(y_test, y_pred.round()))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred.round()))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Accuracy: 0.7
Classification Report:
               precision    recall  f1-score   support

           0       0.70      1.00      0.82         7
           1       0.00      0.00      0.00         3

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10

Confusion Matrix:
 [[7 0]
 [3 0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"test loss: {test_loss}")
print(f"test accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.7000 - loss: 0.6531
test loss: 0.6531308889389038
test accuracy: 0.699999988079071


In [18]:
# cümle sınıflandırma
def classify_sentence(sentence):
  seq = tokenizer.texts_to_sequences([sentence])
  padded_seq = pad_sequences(seq, maxlen=maxlen)

  prediction = model.predict(padded_seq)

  prediction_class = (prediction > 0.5).astype(int)
  label = "positive" if prediction_class[0][0] == 1 else "negative"
  print(f"Prediction: {label}")

  return prediction_class

In [19]:
sentence = "Yemekler çok abartılmış"
result = classify_sentence(sentence)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
Prediction: negative
[[0]]
